# Fine tuning on synthetic function-calling data

This notebook covers how to fine-tune to increase function calling accuracy and reliability.\
You can find more information on function calling [here](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb), 
and on fine tuning [here](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb)

For context, from the function calling notebook above:
> `functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

Function calling is a very powerful tool when it functions as intended. However, we have seen that as the number of \
functions increases, and the complexity of the task at hand increases, function calling becomes less accurate (e.g.: more hallucinated
invocations, and incorrect invocations).\
Before fine tuning for function calling, it's best to begin with:
- Improvements to the function definitions. Make them more clear, and more distinct from one another.
- Experiment with prompt engineering: often a more detailed prompt can help the model call the correct function.\

*If* the steps above fail to improve function calling to a satisfactory level, then you can try fine tuning for function calling.

### Overview

This notebook contains three sections
- **Assessing baseline function calling performance:** Evaluating an out-of-the-box `gpt-3.5-turbo` model on our given function (let's assume that for latency + cost reasons we cannot use `gpt-4` for a drone copilot)
- **Generating synthetic data:** Using `gpt-4` to create 'golden' set of prompts and function invocations to use as training data
- **Fine-tuning**: Running the fine tuning job, and evaluating the fine-tuned model


Note: *This notebook provides an example of how to create synthetic training data for fine tuning for function calling given just a list of functions. While real-world production test evals are preferable, this method produces strong results and can be used in conjuction with real-world training data.*

# Getting baseline function calling performance

In [1]:
# !pip install tenacity
# !pip insta openai
# !pip install typing


In [2]:
import openai
import numpy as np
import json
import os
from openai import OpenAI
import itertools
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import Any, Dict, List, Generator
import ast
client = OpenAI()


### Utilities

Let's define utility functions for making calls to the Chat Completions API, one to get the completion and one to get the function call.

In [3]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(5))
def get_chat_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=1.0,
    stop=None,
    functions=None,
) -> str:
    params = {
        'model': model,
        'messages': messages,
        'max_tokens': max_tokens,
        'temperature': temperature,
        'stop': stop,
    }
    if functions:
        params['functions'] = functions

    completion = client.chat.completions.create(**params)
    return completion.choices[0].message


### Baseline testing

Let's build an intelligent drone co-pilot. We want to be able to give the co-pilot commands, and have it either call the function
for that command, or deny that request if the command is unfeasible.
We can first define a system prompt for the copilot.

In [4]:
DRONE_SYSTEM_PROMPT = """You are an intelligent AI that controls a drone. Given a command or request from the user,
call one of your functions to complete the request. If the request cannot be completed by your available functions, call the reject_request function.
If the request is ambiguous or unclear, reject the request."""


Now let's define functions for all of the actions the copilot can take.

In [5]:
function_list = [
    {
        "name": "takeoff_drone",
        "description": "Initiate the drone's takeoff sequence.",
        "parameters": {
            "type": "object",
            "properties": {
                "altitude": {
                    "type": "integer",
                    "description": "Specifies the altitude in meters to which the drone should ascend."
                }
            },
            "required": ["altitude"]
        }
    },
    {
        "name": "land_drone",
        "description": "Land the drone at its current location or a specified landing point.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "enum": ["current", "home_base", "custom"],
                    "description": "Specifies the landing location for the drone."
                },
                "coordinates": {
                    "type": "object",
                    "description": "GPS coordinates for custom landing location. Required if location is 'custom'."
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "control_drone_movement",
        "description": "Direct the drone's movement in a specific direction.",
        "parameters": {
            "type": "object",
            "properties": {
                "direction": {
                    "type": "string",
                    "enum": ["forward", "backward", "left", "right", "up", "down"],
                    "description": "Direction in which the drone should move."
                },
                "distance": {
                    "type": "integer",
                    "description": "Distance in meters the drone should travel in the specified direction."
                }
            },
            "required": ["direction", "distance"]
        }
    },
    {
        "name": "set_drone_speed",
        "description": "Adjust the speed of the drone.",
        "parameters": {
            "type": "object",
            "properties": {
                "speed": {
                    "type": "integer",
                    "description": "Specifies the speed in km/h."
                }
            },
            "required": ["speed"]
        }
    },
    {
        "name": "control_camera",
        "description": "Control the drone's camera to capture images or videos.",
        "parameters": {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["photo", "video", "panorama"],
                    "description": "Camera mode to capture content."
                },
                "duration": {
                    "type": "integer",
                    "description": "Duration in seconds for video capture. Required if mode is 'video'."
                }
            },
            "required": ["mode"]
        }
    },
    {
        "name": "control_gimbal",
        "description": "Adjust the drone's gimbal for camera stabilization and direction.",
        "parameters": {
            "type": "object",
            "properties": {
                "tilt": {
                    "type": "integer",
                    "description": "Tilt angle for the gimbal in degrees."
                },
                "pan": {
                    "type": "integer",
                    "description": "Pan angle for the gimbal in degrees."
                }
            },
            "required": ["tilt", "pan"]
        }
    },
    {
        "name": "set_drone_lighting",
        "description": "Control the drone's lighting for visibility and signaling.",
        "parameters": {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["on", "off", "blink", "sos"],
                    "description": "Lighting mode for the drone."
                }
            },
            "required": ["mode"]
        }
    },
    {
        "name": "return_to_home",
        "description": "Command the drone to return to its home or launch location.",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "set_battery_saver_mode",
        "description": "Toggle battery saver mode.",
        "parameters": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["on", "off"],
                    "description": "Toggle battery saver mode."
                }
            },
            "required": ["status"]
        }
    },
    {
        "name": "set_obstacle_avoidance",
        "description": "Configure obstacle avoidance settings.",
        "parameters": {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["on", "off"],
                    "description": "Toggle obstacle avoidance."
                }
            },
            "required": ["mode"]
        }
    },
    {
        "name": "set_follow_me_mode",
        "description": "Enable or disable 'follow me' mode.",
        "parameters": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["on", "off"],
                    "description": "Toggle 'follow me' mode."
                }
            },
            "required": ["status"]
        }
    },
    {
        "name": "calibrate_sensors",
        "description": "Initiate calibration sequence for drone's sensors.",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
    {
        "name": "set_autopilot",
        "description": "Enable or disable autopilot mode.",
        "parameters": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["on", "off"],
                    "description": "Toggle autopilot mode."
                }
            },
            "required": ["status"]
        }
    },
    {
        "name": "configure_led_display",
        "description": "Configure the drone's LED display pattern and colors.",
        "parameters": {
            "type": "object",
            "properties": {
                "pattern": {
                    "type": "string",
                    "enum": ["solid", "blink", "pulse", "rainbow"],
                    "description": "Pattern for the LED display."
                },
                "color": {
                    "type": "string",
                    "enum": ["red", "blue", "green", "yellow", "white"],
                    "description": "Color for the LED display. Not required if pattern is 'rainbow'."
                }
            },
            "required": ["pattern"]
        }
    },
    {
        "name": "set_home_location",
        "description": "Set or change the home location for the drone.",
        "parameters": {
            "type": "object",
            "properties": {
                "coordinates": {
                    "type": "object",
                    "description": "GPS coordinates for the home location."
                }
            },
            "required": ["coordinates"]
        }
    },
    {
        "name": "reject_request",
        "description": "Use this function if the request is not possible.",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    },
]


For starters, let's see how function calling performs with some straight forward feasible prompts, and then one obviously impossible request which call the 'reject_request' function.

In [6]:
straightforward_prompts = ['Land the drone at the home base',
 'Take off the drone to 50 meters',
 'change speed to 15 kilometers per hour',
  'turn into an elephant!']


In [7]:
for prompt in straightforward_prompts:
  messages = []
  messages.append({"role": "system", "content": DRONE_SYSTEM_PROMPT})
  messages.append({"role": "user", "content": prompt})
  completion = get_chat_completion(model="gpt-3.5-turbo",messages=messages,functions=function_list)
  print(prompt)
  print(completion.function_call,'\n')


Land the drone at the home base
FunctionCall(arguments='{\n  "location": "home_base"\n}', name='land_drone') 

Take off the drone to 50 meters
FunctionCall(arguments='{\n  "altitude": 50\n}', name='takeoff_drone') 

change speed to 15 kilometers per hour
FunctionCall(arguments='{\n  "speed": 15\n}', name='set_drone_speed') 

turn into an elephant!
FunctionCall(arguments='{}', name='reject_request') 



Nice! The model performs quite well with these requests. Now let's try some more difficult requests: requests that are *almost* feasible and are drone-related, but that the drone cannot actually do, and the pilot should reject.

In [8]:
challenging_prompts = ['Play pre-recorded audio message',
 'Initiate live-streaming on social media',
 'Scan environment for heat signatures',
 'Enable stealth mode',
 "Change drone's paint job color"]


In [9]:
for prompt in challenging_prompts:
  messages = []
  messages.append({"role": "system", "content": DRONE_SYSTEM_PROMPT})
  messages.append({"role": "user", "content": prompt})
  completion = get_chat_completion(model="gpt-3.5-turbo",messages=messages,functions=function_list)
  print(prompt)
  try:
    print(completion.function_call)
    print('\n')
  except:
    print(completion.content)
    print('\n')


Play pre-recorded audio message
FunctionCall(arguments='{}', name='reject_request')


Initiate live-streaming on social media
FunctionCall(arguments='{\n"mode": "video",\n"duration": 0\n}', name='control_camera')


Scan environment for heat signatures
None


Enable stealth mode
FunctionCall(arguments='{\n  "mode": "off"\n}', name='set_drone_lighting')


Change drone's paint job color
FunctionCall(arguments='{\n  "pattern": "solid",\n  "color": "blue"\n}', name='configure_led_display')




Now we run into some problems.
The model here should reject all of these requests, as they are impossible given the functions, however instead the model calls functions that are somewhat related to the request, but incorrect. The model sets the camera to video when asked to begin 'live streaming to social media', and changes the LED's to blue when asked to 'change the paint color'...\
<br>
In this simple case, more prompt engineering may resolve some of these issues, but for the purpose of this example we will demonstrate how fine tuning can be used to improve performance. Additionally, while this case is relatively straightforward, as the number of and complexity of the functions increases, fine tuning becomes more and more impactful.

# Generating synthetic data

### Helper functions

We want to generate every invocation of every function, so that we have
full coverage of all potential invocations to create synthetic data for. Then, we will use `gpt-4` to come up with prompts that would call each invocation, and we will use that prompt - function invocation pair as training data.

Generating every invocation for a function with fixed enums is more simple, but for a function such as
  `control_gimbal` we need to set the `tilt` and `pan` integer values, so to generate those synthetic invocations we will first set a placeholder, and then later use `gpt-4` to come up with reasonable values.

In [10]:
placeholder_int = 'fill_in_int'
placeholder_string = 'fill_in_string'


The functions below take in all the functions from the function list, and look 
at all the potential invocations of those functions given each function's parameters.
The functions also account for `required` parameters, so that all the invocations
are actually feasible.

In [11]:
def generate_permutations(params: Dict[str, Dict[str, Any]]) -> Generator[Dict[str, Any], None, None]:
    """
    Generates all possible permutations for given parameters.

    :param params: Parameter dictionary containing required and optional fields.
    :return: A generator yielding each permutation.
    """

    # Extract the required fields from the parameters
    required_fields = params.get('required', [])

    # Generate permutations for required fields
    required_permutations = generate_required_permutations(params, required_fields)

    # Generate optional permutations based on each required permutation
    for required_perm in required_permutations:
        yield from generate_optional_permutations(params, required_perm)


def generate_required_permutations(params: Dict[str, Dict[str, Any]], required_fields: List[str]) -> List[Dict[str, Any]]:
    """
    Generates permutations for the required fields.

    :param params: Parameter dictionary.
    :param required_fields: List of required fields.
    :return: A list of permutations for required fields.
    """

    # Get all possible values for each required field
    required_values = [get_possible_values(params, field) for field in required_fields]

    # Generate permutations from possible values
    return [dict(zip(required_fields, values)) for values in itertools.product(*required_values)]


def generate_optional_permutations(params: Dict[str, Dict[str, Any]], base_perm: Dict[str, Any]) -> Generator[Dict[str, Any], None, None]:
    """
    Generates permutations for optional fields based on a base permutation.

    :param params: Parameter dictionary.
    :param base_perm: Base permutation dictionary.
    :return: A generator yielding each permutation for optional fields.
    """

    # Determine the fields that are optional by subtracting the base permutation's fields from all properties
    optional_fields = set(params['properties']) - set(base_perm)

    # Iterate through all combinations of optional fields
    for field_subset in itertools.chain.from_iterable(itertools.combinations(optional_fields, r) for r in range(len(optional_fields) + 1)):

        # Generate product of possible values for the current subset of fields
        for values in itertools.product(*(get_possible_values(params, field) for field in field_subset)):

            # Create a new permutation by combining base permutation and current field values
            new_perm = {**base_perm, **dict(zip(field_subset, values))}

            yield new_perm


def get_possible_values(params: Dict[str, Dict[str, Any]], field: str) -> List[Any]:
    """
    Retrieves possible values for a given field.

    :param params: Parameter dictionary.
    :param field: The field for which to get possible values.
    :return: A list of possible values.
    """

    # Extract field information from the parameters
    field_info = params['properties'][field]

    # Based on the field's type or presence of 'enum', determine and return the possible values
    if 'enum' in field_info:
        return field_info['enum']
    elif field_info['type'] == 'integer':
        return [placeholder_int]
    elif field_info['type'] == 'string':
        return [placeholder_string]
    elif field_info['type'] == 'boolean':
        return [True, False]
    elif field_info['type'] == 'array' and 'enum' in field_info['items']:
        enum_values = field_info['items']['enum']
        all_combinations = [list(combo) for i in range(1, len(enum_values) + 1) for combo in itertools.combinations(enum_values, i)]
        return all_combinations
    return []


### Let's generate every invocation for every function first

Prompts:

In [12]:
INVOCATION_FILLER_PROMPT = """
1) Input reasonable values for 'fill_in_string' and 'fill_in_int' in the invocation here: {invocation}. Reasonable values are determined by the function definition. Use the
the entire function provided here :{function} to get context over what proper fill_in_string and fill_in_int values would be.
Example:

Input: invocation: {{
    "name": "control_camera",
    "arguments": {{
      "mode":"video",
      "duration":"fill_in_int"
    }}
}},
function:{function}

Output: invocation: {{
    "name": "control_camera",
    "arguments": {{
      "mode":"video",
      "duration": 30
    }}
}}


MAKE SURE output is just a dictionary with keys 'name' and 'arguments', no other text or response.

Input: {invocation}
Output:
"""


COMMAND_GENERATION_PROMPT= """
You are to output 2 commands, questions or statements that would generate the inputted function and parameters.
Please make the commands or questions natural, as a person would ask, and the command or questions should be varied and not repetitive.
It should not always mirror the exact technical terminology used in the function and parameters, rather reflect a conversational and intuitive request.
For instance, the prompt should not be 'turn on the dome light', as that is too technical, but rather 'turn on the inside lights'.
Another example, is the prompt should not be 'turn on the HVAC', but rather 'turn on the air conditioning'. Use language a normal driver would use, even if
it is technically incorrect but colloquially used.

RULES: ALWAYS put a backwards slash before an apostrophe or single quote '. For example, do not say don't but say don\'t.
Prompts MUST be in double quotes as well.

Example

Input: {{'name': 'calibrate_sensors','arguments': {{}}'' }}
Prompt: ["The sensors are out of whack, can you reset them", "The calibration of the drone is off, fix it please!"]

Input: {{'name': 'set_autopilot','arguments': {{'status': 'off'}}}}
Prompt: ["OK, I want to take back pilot control now","Turn off the automatic pilot I'm ready control it"]

Input: {invocation}
Prompt:
"""


In the below snippet, we generate the invocation of each function except for the rejection_request function.\
To perform effective fine-tuning we need correctly labeled data. We could manually come up with examples and label the data,\
or we can generate synthetic data with the help of `gpt-4` <br>
Empirically, `gpt-4` needs a bit more help to get good realistic examples of prompts that would generate the reject_request function, so we'll do that next...

In [13]:
input_objects = []
all_but_reject = [f for f in function_list if f.get('name') != 'reject_request']

for function in all_but_reject:
    func_name = function["name"]
    params = function["parameters"]
    for arguments in generate_permutations(params):
      if any(val in arguments.values() for val in ['fill_in_int', 'fill_in_str']):
          input_object = {
              "name": func_name,
              "arguments": arguments
          }
          messages = [{"role": "user", "content": INVOCATION_FILLER_PROMPT.format(invocation=input_object,function=function)}]
          input_object = get_chat_completion(model='gpt-4',messages=messages, max_tokens = 200,temperature=.1).content
      else:
          input_object = {
              "name": func_name,
              "arguments": arguments
          }

      input_objects.append(input_object)


Now that we have all the invocations, let's use `gpt-4` to generate prompts that would result in those invocations

In [ ]:
def create_commands(invocation_list):
    example_list = []
    for i, invocation in enumerate(invocation_list):
        print(f'\033[34m{np.round(100*i/len(invocation_list),1)}% complete\033[0m')
        print(invocation)

        # Format the prompt with the invocation string
        request_prompt = COMMAND_GENERATION_PROMPT.format(invocation=invocation)

        messages = [{"role": "user", "content": f"{request_prompt}"}]
        completion = get_chat_completion(messages,temperature=0.8).content
        command_dict = {
            "Input": invocation,
            "Prompt": completion
        }
        example_list.append(command_dict)
    return example_list


In [ ]:
training_examples_unformatted = create_commands(input_objects)


Now let's format the training examples properly. For more documentation on the proper training data formatting for fine tuning for function calling, see here: https://platform.openai.com/docs/guides/fine-tuning/fine-tuning-examples


In [ ]:
training_examples = []
for prompt in training_examples_unformatted:
    #adjust formatting for training data specs
    try:
        prompt["Input"] = ast.literal_eval(prompt["Input"])
    except:
        continue
    prompt['Input']['arguments']=json.dumps(prompt['Input']['arguments'])
    for p in prompt['Prompt']:
        training_examples.append({"messages": [{"role":"system","content":DRONE_SYSTEM_PROMPT
                                        },{"role":"user","content": p},
                            {"role":"assistant","function_call": prompt['Input']}],
                            "functions":function_list})


Now, back to the rejection function. Let's generate some prompts that are *nearly* possible, but should result in the `decline_request` function being called. To do so, we queried `gpt-4` asking for requests that are related to, but not quite possible with, the given list of functions. 

In [ ]:
reject_list = ['Translate broadcast message to another language',
'Automatically capture photos when face is detected',
'Detect nearby drones',
'Measure wind resistance',
'Capture slow motion video',
"Adjust drone's altitude to ground level changes",
'Display custom message on LED display',
"Sync drone's time with smartphone",
'Alert when drone travels out of designated area',
'Detect moisture levels',
'Automatically follow GPS tagged object',
'Toggle night vision mode',
'Maintain current altitude when battery is low',
'Decide best landing spot using AI',
"Program drone's route based on wind direction"]


In [ ]:
reject_training_list = []
for prompt in reject_list:

    #Adjust formatting
    reject_training_list.append({"messages": [{"role":"system","content":DRONE_SYSTEM_PROMPT
                                    },{"role":"user","content": prompt},
                        {"role":"assistant","function_call": {"name": "reject_request","arguments": "{}"}}],
                        "functions":function_list})


Now combine all the training examples together

In [ ]:
training_list_total = training_examples+reject_training_list


In [ ]:
training_file = 'data/drone_training.jsonl'
with open(training_file, 'w') as f:
    for item in training_list_total:
        json_str = json.dumps(item)
        f.write(f'{json_str}\n')


# Fine tuning

Finally, we can kick off the fine-tuning job

In [ ]:
if __name__ == "__main__":
    file = client.files.create(
        file=open(training_file, "rb"),
        purpose="fine-tune",
    )
    file_id = file.id
    print(file_id)
    ft = client.fine_tuning.jobs.create(
        # model="gpt-4-0613",
        model="gpt-3.5-turbo",
        training_file=file_id,
)


# Evaluations

Great! We trained a fine-tuned model for function calling. Let's see how it does on our evaluation set for prompts that the drone assistant
should automatically reject.

In [ ]:
for eval_question in challenging_prompts:
  messages = []
  messages.append({"role": "system", "content": DRONE_SYSTEM_PROMPT})
  messages.append({"role": "user", "content": eval_question})
  completion = get_chat_completion(model="ft:gpt-3.5-turbo-0613:openai-internal::8DloQKS2",messages=messages,functions=function_list)
  print(eval_question)
  print(completion.function_call,'\n')


Great! While the original model only rejected 1 of the 5 requests, the fine tuned model rejected all 5 requests.

### Conclusion

Congratulations! You are now ready to fine tune your model for function calling. We can't wait to see what you build.